## *Estructura:*

## Lectura del historico del mes:

El proceso comienza cargando todos los ficheros de alarmas del mes en curso. Sí hay ficheros los lee y guarda los msisdn en un diccionario como clave. si no existe ningun fichero se genera un dataframe vacio con la misma estructura que en el historico. El sentido de cargar el pasado antes de analizar el presente es porque no se quiere reportar msisdn en un mes que ya se hayan reportado.

## Lectura Fichero_detalle_actual y General Zonas

Una vez se lee el fichero_detalle_actual se le aplica un left join en el sentido (Fichero_detalle_actual <- General Zonas) con el fin de aportarle al msisdn la información del operador con el previo tratamiento y aplicación de lógicas al general zonas. Tras esto aparece una posible casuística y como es un proceso sensible se pregunta a la columna "COD_CLIENTE_PADRE" si tiene NaN, ya que han podido aparecer tras el cruce. (el operador se metería a mano tras consultar el msisdn en ATENEA)

Con la finalización de este punto y el posterior agrupado por ['msisdn','NAME','COD_CLIENTE_PADRE','recordType'] llegamos al siguiente escenario planteado en el apartado anterior:

-> df_historico *empty*: en el dataframe actual se genera un columna "FLAG" donde todos las filas son False.(False será la condición necesaria para que un msisdn sea candidato a alarma)
            
-> df_historico *not empty*: se genera un diccionario con los msisdn como clave y se itera sobre la columna columna "msisdn" del dataframe actual para preguntarle si está en el diccionario   historico, de tal manera de que si lo encuentra en el diccionario devuelve un True (msisdn que no es candidato a enviar en alarma) y sino False.

Con la categorización de lo que es candidato a enviar se realiza un filtro donde se seleciona todo lo que sea False.

El siguiente punto sería añadir los directorios realizando un LEFT JOIN sobre el operador para incorporar los emails. Y de la misma manera que se realizó anteriormente una correción sobre el merge, aquí lo volvemos a repetir en caso de que haya alguno campo de la columna "MAIL" como NaN.

## Función enviar_correos:

En esta función se crea la configuración al servidor SMTP, incorpora las credencias del email a traves de un fichero yaml y se aplican una seríe de lógicas para que configure el campo "FROM" y "SUBJECT" en función del operador. Además, hay un control mediante los except de los errores que puedan ocurrir durante el envio. En este punto se guarda en una lista todos los operadores que han sufrido un error en los envios de tal forma que cuando se termine de enviar todos los correos se pueda analizar y reenviar los operdores con error

# Libreria

In [ ]:
from Utils import _get_general_zonas, get_email
from modulodirectorio import get_path

import pathlib
import pandas as pd
import os
import glob
import shutil
import warnings
from datetime import datetime
import re
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import quopri
import sys

warnings.filterwarnings('ignore')


fecha_actual = datetime.now()
año = fecha_actual.year
mes = fecha_actual.month
dia = fecha_actual.day
date_str = str(año)+'_'+str(mes)+'_'+str(dia)

mes_revision = '202307'  # TODO considerar obtener el mes a partir de la fecha actual, o si acaso, desde el fichero de detalle (para lidiar con cambios de mes)

# dir reference

In [ ]:
dir_detalle = get_path('dir_detalle')
dir_historico_procesed = get_path('dir_historico_procesed')
directorio_base = get_path('directorio_base')
directorio_crudo = get_path('directorio_crudo')
directorio_email = get_path('directorio_email')

# Lectura retarificaciones

In [ ]:
def lectura_retarificaciones(mes, base_dir):
    file_key = str(mes_revision)+' retarificación'

    for nameCSV in glob.glob(os.path.join(base_dir ,"*.xlsx")):
        file_name=str(nameCSV)
        if file_key in file_name:
            retarificaciones = pd.read_excel(file_name)
            
    retarificaciones['msisdn'] = retarificaciones['msisdn'].astype(str)
    retarificaciones['variacion precio'] = retarificaciones['importe ocs'] - retarificaciones['importe correcto']
    retarificaciones = retarificaciones.groupby(['msisdn']).sum().reset_index()
    retarificaciones = retarificaciones[['msisdn','variacion precio']]
    retarificaciones = retarificaciones[retarificaciones['variacion precio'] > 0] # Nos quedamos solo con las aminoraciones de importe
    return retarificaciones


def check_retarificaciones(mes, base_dir, df):
    # primero averiguamos si se han hecho las retarificaciones
    while True:
        retarificaciones_realizadas = input('¿Se han realizado retarificaciones en el momento de envío de las alarmas? (s/n): ')

        if retarificaciones_realizadas.lower() != 's' and retarificaciones_realizadas.lower() != 'n':
            print('Respuesta inválida. Por favor, ingresa "s" o "n".')
        else: break
    
    # si no se han hecho, hemos terminado
    if retarificaciones_realizadas.lower() == 'n':
        df['FLAG'] = False
        return df

    # en caso afirmativo, comprobamos los importes
    df = df.merge(lectura_retarificaciones(mes_revision, base_dir), how = 'left', on = 'msisdn')
    df['variacion precio'] = df['variacion precio'].fillna(0)
    df['Importe Acumulado'] = df['Importe Acumulado'].astype(float)
    df['Porcentaje retarificacion'] = (df['variacion precio']/df['Importe Acumulado'])*100
    df['FLAG'] = df['Porcentaje retarificacion'].map(lambda x: x >= 75)
    
    print('Los msisdn con alguna retarifacion son:')
    display(df[df['Porcentaje retarificacion'] > 0])
    print('')
    print('Aquellos con retarificaciones significativas son')
    display(df[df['FLAG']])
    
    return df

# Lectura dir email

In [ ]:
df_contactos = pd.read_csv(directorio_email, encoding='ansi', sep=';')
df_contactos['Cod Operador'] = df_contactos['Cod Operador'].astype(str)

if df_contactos.isnull().values.any(): raise ValueError('Valores faltantes en el fichero de contactos.')

# Lectura del historico del mes:

In [ ]:
# obtenemos todos los históricos que contienen yyyymm, para unificarlos en uno

files = []

for nameCSV in glob.glob(os.path.join(dir_historico_procesed, '*.csv')):
    name = str(nameCSV)
    if mes_revision in name and os.path.isfile(name):
        name = pd.read_csv(name, delimiter=';', encoding='ansi')
        files.append(name)

if files:
    df_historico = pd.concat(files)
else:  # la lista está vacía, esto ocurre en mes nuevo
    df_historico = pd.DataFrame(
        columns=['msisdn', 'Cod Operador', 'Tipo de Evento', 'Nº Eventos', 'Suma Minutos', 'Importe en el mes', 'Importe Acumulado', 'Suma MB','billing period']) 

msisdn_previos = df_historico['msisdn'].unique().tolist()

# Lectura Fichero_detalle_actual y General Zonas

In [ ]:
# FICHERO DETALLE-------------------------------------------------------------------------

# primero, verificamos que sólo exista un fichero en el directorio de detalle
file_list = glob.glob(os.path.join(dir_detalle, '*.txt'))
if not file_list: raise ValueError(f'No hay fichero en el directorio de detalle: {dir_detalle}')
if len(file_list) > 1: raise ValueError(f'Existen múltiples ficheros en el directorio de detalle: {dir_detalle}')

filepath = file_list[0]
filename = os.path.basename(filepath)

# ahora, verificamos que el nombre del fichero tenga sentido
if not bool(re.match(r'^{}(\d{{2}}).*'.format(mes_revision), filename)):
    raise ValueError('El fichero no tiene un nombre válido')

# leemos el fichero y lo movemos al histórico de crudos
df_detalle = pd.read_csv(filepath, delimiter=';', decimal = ',')
shutil.move(filepath, directorio_crudo)


# GENERAL ZONAS---------------------------------------------------------------------------
zonas = _get_general_zonas()  # TODO considerar utilizar simplemente el más actualizado, así no se requiere el cambio del nombre
g_zonas = pd.read_csv(zonas,sep=';')
g_zonas = g_zonas.sort_values('FEC_ALTA_ABONADO', ascending=True)
g_zonas = g_zonas.drop_duplicates(subset='ES_MOVIL_FIXED', keep='first')
g_zonas = g_zonas[['ES_MOVIL_FIXED','COD_CLIENTE_PADRE','OPERADOR','NAME']]
g_zonas = g_zonas.rename(columns={'ES_MOVIL_FIXED': 'msisdn', 'COD_CLIENTE_PADRE': 'COD OPERADOR'})
g_zonas['COD OPERADOR'] = g_zonas['COD OPERADOR'].astype(str)


# LEFT JOIN (Fichero_detalle_actual <- General Zonas)-------------------------------------
df_detalle_clean = df_detalle.merge(g_zonas, how = 'left', on='msisdn')


# COMPROBACION DE LOS NULOS --------------------------------------------------------------

# vamos a iterar sobre los nulos, controlando tanto sus posiciones en el df, como el progreso en la entrada de datos
null_positions = [i for i, value in enumerate(df_detalle_clean['COD OPERADOR']) if pd.isnull(value)]
num_null_values = len(null_positions)

for i, pos in enumerate(null_positions, start=1):
    msisdn = df_detalle_clean.at[pos, 'msisdn']
    correct_value = str(int(input(f'Ingrese el código del operador en ATENEA para el MSISDN {msisdn} ({i}/{num_null_values})')))
    df_detalle_clean.at[pos, 'COD OPERADOR'] = correct_value

# realizamos los agrupados, y preparamos el formato

df_detalle_clean = df_detalle_clean.groupby(
    by=['msisdn', 'NAME','COD OPERADOR','recordType','priceAcu']).agg(
        {'units': sum, 'minutes': sum, 'price': sum, 'mb': sum}).reset_index()

df_detalle_clean = df_detalle_clean.rename(
    columns = {'COD OPERADOR': 'Cod Operador',
               'NAME': 'Operador',
               'priceAcu': 'Importe Acumulado',
               'recordType': 'Tipo de Evento',
               'units': 'Nº Eventos',
               'minutes': 'Suma Minutos',
               'mb': 'Suma MB',
               'price': 'Importe en el mes'})

df_detalle_clean['Periodo'] = mes_revision
df_detalle_clean['msisdn'] = df_detalle_clean['msisdn'].astype(str)

# como los inicios del mes siempre va haber un dataframe vacio, todo lo que haya en el fichero es envio. Así nos evitamos errores en el merge.
if not df_historico.empty: df_detalle_clean['FLAG'] = df_detalle_clean['msisdn'].map(lambda x: x in msisdn_previos)
else: df_detalle_clean['FLAG'] = False # como está vacio se envia todo

df_detalle_clean = check_retarificaciones(mes_revision, base_dir, df_detalle_clean)

# nos quedamos con aquellos sin retarificaciones masivas
df_detalle_clean = df_detalle_clean[~df_detalle_clean['FLAG']]
df_detalle_clean.to_csv(get_path('dir_historico_procesed') / (filename[:-4]+'.csv'), index = False, sep = ';', encoding='ansi')

# para cruzar los contactos, primero trabajamos a nivel operador
df_mail = pd.DataFrame(df_detalle_clean['Cod Operador'].drop_duplicates())
df_mail = df_mail.merge(df_contactos, how = 'left', on ='Cod Operador')

# añadimos aquellos que falten
null_positions = [i for i, value in enumerate(df_mail['MAIL']) if pd.isnull(value)]
num_null_values = len(null_positions)

for i, pos in enumerate(null_positions, start=1):
    cod = df_mail.at[pos, 'Cod Operador']
    correct_value = str(input(f'Ingrese el email de contacto para el operador {cod} ({i}/{num_null_values})'))
    # añadimos los nuevos contactos en el df de contactos
    new_row = {'Cod Operador': cod, 'MAIL': correct_value}
    df_contactos = df_contactos.append(new_row, ignore_index=True)

# guardamos el fichero de contactos
df_contactos.to_csv(directorio_email, encoding='ansi', sep=';', index=False)

# cruzamos a nivel MSISDN
df_final = df_detalle_clean.merge(df_contactos, how='left', on='Cod Operador')

In [ ]:
def enviar_correo(destinatario, archivo_adjunto, operador, date_str, email, passw):

    destinatario = destinatario.split(' ')
    
    if len(destinatario) <= 1 and any('' == email for email in destinatario):
        print(destinatario)
        return False
    
    destinatario.append('ejemplo@:).es')
    
    # Configurar el servidor SMTP y el mensaje
    smtp_server = 'smtp-mail.outlook.com'
    smtp_port = 587
    smtp_username = email
    smtp_password = passw
    body = '''CUERPO DEL CORREO
'''

    # Crear el objeto del correo electrónico
    msg = MIMEMultipart()
    msg['From'] = 'ejemplo@:).es'
    recipients = ', '.join(destinatario)

    # gestionar caso recipiente ''
    msg['To'] = recipients

    msg['Subject'] = 'Alarma de consumo superior a 49 € - ' + operador
    msg.attach(MIMEText(body, 'plain'))
    # Adjuntar el archivo al correo electrónico
    with open(archivo_adjunto, 'rb') as file:
        attachment = MIMEBase('application', 'octet-stream')
        attachment.set_payload(file.read())
        encoders.encode_base64(attachment)
        attachment.add_header('Content-Disposition', 'attachment', filename= date_str + '_' +operador + '.xlsx')
        msg.attach(attachment)

    # Enviar el correo electrónico
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(smtp_username, smtp_password)
        server.send_message(msg)

In [ ]:
email, passw = get_email()

operadores = df_final['Cod Operador'].unique()

error_files = []

c = 0
c_e = 0

for operador in operadores:
    # Crear DataFrame para el operador actual
    df_operador = df_final[df_final['Cod Operador'] == operador].copy()
    destinatario = df_operador['MAIL'].iloc[0]

    operador_name = df_operador['Operador'].iloc[0]
    df_operador = df_operador[
        ['msisdn', 'Cod Operador', 'Tipo de Evento', 'Nº Eventos', 'Suma Minutos', 'Importe en el mes', 'Importe Acumulado', 'Suma MB']]
    directorio_operador = os.path.join(directorio_base, operador)
    
    if not os.path.exists(directorio_operador):
        os.makedirs(directorio_operador)
    
    archivo_csv = os.path.join(directorio_base, operador)+'/'+date_str+'.xlsx'
    df_operador.to_excel(os.path.join(directorio_base, operador)+'/'+date_str+'.xlsx', index=False, encoding= 'ansi')
    
    try:
        enviar_correo(destinatario, archivo_csv, operador_name, date_str, email, passw)
        c+=1
        print(f'Correo {c} enviado satisfactoriamente -> {operador}')
        
    except:  
        c_e+=1  
        error_files.append(operador)
        print(f'Correo {c_e} error en el envio -> {operador}')
       
print('Errores:')
print(error_files)

In [ ]:

# Reenvio de los correos con la rectificación del email 

for operador in error_files:
    
    # Crear DataFrame para el operador actual
    df_operador = prueba[prueba['Cod Operador'] == operador].copy()
    destinatario = df_operador['MAIL'].iloc[0]
    operador_name = df_operador['Operador'].iloc[0]
    df_operador = df_operador[['msisdn', 'Cod Operador', 'Tipo de Evento', 'Nº Eventos', 'Suma Minutos', 'Importe en el mes', 'Importe Acumulado', 'Suma MB']]
    directorio_operador = os.path.join(directorio_base, operador)
    
    if not os.path.exists(directorio_operador):
        os.makedirs(directorio_operador)
    
    print(f"El directorio '{directorio_operador}' ya existe.")
    archivo_csv = os.path.join(directorio_base, operador)+'/'+date_str+'.xlsx'
    
    df_operador.to_excel(os.path.join(directorio_base, operador)+'/'+date_str+'.xlsx', index=False, encoding= 'ansi')
    if not enviar_correo(destinatario, archivo_csv, operador_name, date_str):
        error_files.append(operador)
        
       

print(error_files)